# 公開鍵暗号アルゴリズム RSAとPython

In [1]:
## create the clean environment
import gc
import matplotlib.pyplot as plt

def clear_all():
    # Clears all the variables from the workspace
    gl = globals().copy()
    for var in gl:
        if var in clean_env_var: continue
        del globals()[var]
    # Garbage collection:
    gc.collect()

def close_plots():
  my_plots = plt.get_fignums()
  for j in my_plots:
    plt.close(plt.figure(j))

clean_env_var = dir()
clean_env_var.append('clean_env_var')

In [2]:
clear_all()

### Hardware

In [3]:
%%bash
system_profiler SPHardwareDataType | grep -E \
"Model Identifier"\|"Processor Name"\|"Processor Speed"\
\|"Number of Processors"\|"Memory:"

      Model Identifier: MacBookPro13,1
      Processor Name: Dual-Core Intel Core i5
      Processor Speed: 2 GHz
      Number of Processors: 1
      Memory: 16 GB


### Python

In [4]:
!python -V

Python 3.7.4


### Import

In [5]:
import numpy as np

## 1. RSAとは

- RSA のアルゴリズムはべき乗と余剰のみで表すことのできる、非常にシンプルなもの
- $\{E, N\}$のペアが公開鍵に相当するとき

$$
\text{暗号文} = \text{平文}^E mod N
$$

- $\{D, N\}$のペアが複合鍵に相当するとき

$$
\text{平文} = \text{暗号文}^E mod N
$$

### RSAの仕組み

<img src = "./fig/RSA_1.jpg">


## 2. RSAがなぜ機能するのか

証明の入る前に合同式の定理のおさらいをする

### 合同式の性質 1

- p を自然数とする
- $\mathbb Z_p \equiv \{0, 1, ..., p-1\}$


$a \equiv b \text{ mod }p$のとき、任意の自然数cについて

$$
\begin{aligned}
a + c &\equiv b + c \ \ (\text{ mod } p)\\
a - c &\equiv b - c  \ \ (\text{ mod } p)\\
ac &\equiv bc  \ \ (\text{ mod } p)\\
a^c &\equiv b^c  \ \ (\text{ mod } p)
\end{aligned}
$$

### 合同式の性質 2

a, pを互いに素な自然数とする。このとき、任意に$b\in [0, p-1]$に選んできたとき、以下の条件を満たす$x\in \mathbb Z_p$はbに対してただ一つしか存在しない：

$$
ax \equiv b \ \ (\text{ mod } p)
$$


### Chinese Remainder Theorem

p, qを互いに素な自然数とする。このとき$x \equiv a \  \ \text{ mod } p$ and $x \equiv a \  \ \text{ mod } q$ならば、$x \equiv a \  \ \text{ mod } pq$

#### 証明

$x \equiv b \text{ mod } pq$、　ただし$b < pq$とする。WTS : $b = a$。

$x \equiv a \  \ \text{ mod } p$より$b \equiv a \  \ \text{ mod } p$。同様に、$b \equiv a \  \ \text{ mod } q$

よって、$b = pt_1 + a = qt_2 + b$とかける。このとき$pt_1 = qt_2$だが、p,qは互いに素なので一番小さい場合でも$b = pq + a$.仮定より、$b < pq$なので $b = a$.


### RSAの復号の証明

$$
m^{k_1k_2} \equiv m \text{ mod } n
$$
を証明すれば良い。chinese remainder theoremより

$$
m^{k_1k_2} \equiv m \text{ mod } p
$$
を証明すれば十分（対称性より）。


- mがpの倍数のとき自明。
- mがpの倍数ではないとき

$k_1k_2 - 1$が$p-1$の倍数になるように設定してあるので、$T\in \mathbb N$

$$
k_1k_2 = 1 + T(p-1)
$$

Then,

$$
m^{k_1k_2}=m\cdot (m^{p-1})^N
$$


フェルマーの小定理とmがpの倍数ではないので

$$
m^{p-1} \equiv 1^N
$$

よって

$$
m^{k_1k_2}=m\cdot (m^{p-1})^N\equiv m\cdot 1^N=m
$$


## 3. RSAアルゴリズムの実装

In [6]:
def key_generator(p, q):
    N = p * q
    L = np.lcm(p - 1, q - 1) # lcmでも良い
    
    for i in range(2, L):
        if np.gcd(i, L) == 1:
            k1 = i
            break
            
    for i in range(2, L):
        if (k1 * i) % L == 1:
            k2 = i
            break
            
    private, public = (k1, N), (k2, N)
    return private, public

def rsa_encoder(text, public_key):
    k1, N = public_key
    plain_unicode = [ord(char) for char in text]
    encrypted_unicode = [pow(i, k1, N) for i in plain_unicode]
    encrypted_text = ''.join(chr(i) for i in encrypted_unicode)
    return encrypted_text

def rsa_decoder(text, private_key):
    k2, N = private_key
    encrypted_unicode = [ord(char) for char in text]
    decrypted_unicide = [pow(i, k2, N) for i in encrypted_unicode]
    decrypted_text = ''.join(chr(i) for i in decrypted_unicode)
    
    return decrypted_text

### Test

In [7]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [8]:
test = """
The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
"""

In [9]:
### 素数の設定
p, q = 499, 929

In [10]:
private, public = key_generator(p, q)
private, public

((5, 463571), (92429, 463571))

In [11]:
encoded_text = rsa_encoder(text = test, public_key = public)
print(encoded_text)

񬶱ᔎ뾄񀺆뾄𜚋𝷏苰񮉼񭰂ꮷ𝷏𜚋𑩽񜼳񭰂ᔎ񪟹𮅭񮉼뾄ꮷ뾄𪳿𲁻񬶱񬶱񉱘뾄෕񞗣ꮷ񪟹苰񞗣𔙸񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋񞗣񃀃𔙸񭰂񕾃񬶱𡆯촥𢧎𔙸񪟹檭񪟹ꮷ񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋񪟹𮅭𢧎𔙸񪟹檭񪟹ꮷ񕾃񬶱񧙭񪟹𮅭𢧎𔙸뾄񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋檭𝷏𮅭𢧎𔙸뾄촥񕾃񬶱񋾡𝷏𮅭𢧎𔙸뾄촥񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋檭𝷏𮅭𢧎𔙸񪟹檭෕ꮷ뾄񋘫񕾃񬶱𩧯𔙸෕ꮷ񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋𜚋뾄𲁻ꮷ뾄񋘫񕾃񬶱񧙭𢧎෕𪳿𲁻뾄񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋񋘫뾄𜚋𲁻뾄񕾃񬶱𢪟뾄෕񋘫෕񜼳񪟹𔙸񪟹ꮷ񭰂檭𝷏񞗣𜚋ꮷ𲁻񕾃񬶱񧙭𢧎뾄檭񪟹෕𔙸檭෕𲁻뾄𲁻෕𪳿뾄𜚋񆩕ꮷ𲁻𢧎뾄檭񪟹෕𔙸뾄𜚋𝷏񞗣񃀃ꮷ𝷏񜼳𪳿뾄෕𴝤ꮷ뾄𪳿񞗣𔙸뾄𲁻񕾃񬶱얧𔙸ꮷ𝷏񞗣񃀃𢧎𪳿෕檭ꮷ񪟹檭෕𔙸񪟹ꮷ񭰂񜼳뾄෕ꮷ𲁻𢧎񞗣𪳿񪟹ꮷ񭰂񕾃񬶱𡆯𪳿𪳿𝷏𪳿𲁻𲁻𝷏񞗣𔙸񋘫𜚋뾄𗲩뾄𪳿𢧎෕𲁻𲁻𲁻񪟹𔙸뾄𜚋ꮷ𔙸񭰂񕾃񬶱𕱧𜚋𔙸뾄𲁻𲁻뾄촥𢧎𔙸񪟹檭񪟹ꮷ𔙸񭰂𲁻񪟹𔙸뾄𜚋檭뾄񋘫񕾃񬶱񕴽𜚋ꮷ뾄苰෕檭뾄𝷏苰෕𮅭񜼳񪟹񃀃񞗣񪟹ꮷ񭰂𑩽𪳿뾄苰񞗣𲁻뾄ꮷ뾄ꮷ뾄𮅭𢧎ꮷ෕ꮷ񪟹𝷏𜚋ꮷ𝷏񃀃񞗣뾄𲁻𲁻񕾃񬶱ᔎ뾄𪳿뾄𲁻𝷏񞗣𔙸񋘫񜼳뾄𝷏𜚋뾄𼅀𼅀෕𜚋񋘫𢧎𪳿뾄苰뾄𪳿෕񜼳𔙸񭰂𝷏𜚋𔙸񭰂𝷏𜚋뾄𼅀𼅀𝷏񜼳𗲩񪟹𝷏񞗣𲁻𙙐෕񭰂ꮷ𝷏񋘫𝷏񪟹ꮷ񕾃񬶱얧𔙸ꮷ𝷏񞗣񃀃ꮷ෕ꮷ𙙐෕񭰂𮅭෕񭰂𜚋𝷏ꮷ񜼳뾄𝷏񜼳𗲩񪟹𝷏񞗣𲁻෕ꮷ苰񪟹𪳿𲁻ꮷ񞗣𜚋𔙸뾄𲁻𲁻񭰂𝷏񞗣񆩕𪳿뾄񡚉񞗣ꮷ檭񕾃񬶱𺪍𝷏𙙐񪟹𲁻񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋𜚋뾄𗲩뾄𪳿񕾃񬶱얧𔙸ꮷ𝷏񞗣񃀃𜚋뾄𗲩뾄𪳿񪟹𲁻𝷏苰ꮷ뾄𜚋񜼳뾄ꮷꮷ뾄𪳿ꮷ෕𜚋񕿎𪳿񪟹񃀃ꮷ񕿎𜚋𝷏𙙐񕾃񬶱񕴽苰ꮷ뾄񪟹𮅭𢧎𔙸뾄𮅭뾄𜚋ꮷ෕ꮷ񪟹𝷏𜚋񪟹𲁻෕𪳿񋘫ꮷ𝷏뾄촥𢧎𔙸෕񪟹𜚋𑩽񪟹ꮷ񆩕𲁻෕񜼳෕񋘫񪟹񋘫뾄෕񕾃񬶱񕴽苰ꮷ뾄񪟹𮅭𢧎𔙸뾄𮅭뾄𜚋ꮷ෕ꮷ񪟹𝷏𜚋񪟹𲁻뾄෕𲁻񭰂ꮷ𝷏뾄촥𢧎𔙸෕񪟹𜚋𑩽񪟹ꮷ𮅭෕񭰂񜼳뾄෕񃀃𝷏𝷏񋘫񪟹񋘫뾄෕񕾃񬶱𺪍෕𮅭뾄𲁻𢧎෕檭뾄𲁻෕𪳿뾄𝷏𜚋뾄𝷏𜚋𴝤񪟹𜚋񃀃񃀃𪳿뾄෕ꮷ񪟹񋘫뾄෕𼅀𼅀𔙸뾄ꮷ񆩕𲁻񋘫𝷏𮅭𝷏𪳿뾄𝷏苰ꮷ𝷏𲁻뾄𼂝񬶱


In [12]:
print(rsa_encoder(text = encoded_text, public_key = private))


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!



In [13]:
test == rsa_encoder(text = encoded_text, public_key = private)

True